In [1]:
import geopandas as gpd


In [2]:
products = gpd.read_parquet("data/products-index.parquet")



In [6]:
products

,uri,product_id,product_label,year_id,year_label,resolution_id,resolution_label,tile_id,tile_label,geometry
0,https://api.agrimetrics.co.uk/tiles/collection...,casi_multispectral_imagery,CASI Multispectral Imagery,2019,2019,1,1m,NU0050,NU05sw,"POLYGON ((405000 650000, 405000 655000, 400000..."
1,https://api.agrimetrics.co.uk/tiles/collection...,lidar_composite_dtm,LIDAR Composite DTM,2022,2022,1,1m,NU0050,NU05sw,"POLYGON ((405000 650000, 405000 655000, 400000..."
2,https://api.agrimetrics.co.uk/tiles/collection...,lidar_composite_dtm,LIDAR Composite DTM,2022,2022,2,2m,NU0050,NU05sw,"POLYGON ((405000 650000, 405000 655000, 400000..."
3,https://api.agrimetrics.co.uk/tiles/collection...,lidar_composite_first_return_dsm,LIDAR Composite First Return DSM,2022,2022,1,1m,NU0050,NU05sw,"POLYGON ((405000 650000, 405000 655000, 400000..."
4,https://api.agrimetrics.co.uk/tiles/collection...,lidar_composite_first_return_dsm,LIDAR Composite First Return DSM,2022,2022,2,2m,NU0050,NU05sw,"POLYGON ((405000 650000, 405000 655000, 400000..."
...,...,...,...,...,...,...,...,...,...,...
193210,https://api.agrimetrics.co.uk/tiles/collection...,lidar_tiles_dsm,LIDAR Tiles DSM,2020,2020,1,1m,TA4515,TA41ne,"POLYGON ((550000 415000, 550000 420000, 545000..."
193211,https://api.agrimetrics.co.uk/tiles/collection...,lidar_tiles_dtm,LIDAR Tiles DTM,2018,2018,1,1m,TA4515,TA41ne,"POLYGON ((550000 415000, 550000 420000, 545000..."
193212,https://api.agrimetrics.co.uk/tiles/collection...,lidar_tiles_dtm,LIDAR Tiles DTM,2020,2020,1,1m,TA4515,TA41ne,"POLYGON ((550000 415000, 550000 420000, 545000..."
193213,https://api.agrimetrics.co.uk/tiles/collection...,lidar_tiles_dsm,LIDAR Tiles DSM,2020,2020,1,1m,TA4520,TA42se,"POLYGON ((550000 420000, 550000 425000, 545000..."


In [3]:
products.product_id.unique()

array(['casi_multispectral_imagery', 'lidar_composite_dtm',
       'lidar_composite_first_return_dsm',
       'lidar_composite_last_return_dsm', 'lidar_point_cloud',
       'lidar_tiles_dsm', 'lidar_tiles_dtm',
       'national_lidar_programme_dsm', 'national_lidar_programme_dtm',
       'national_lidar_programme_first_return_dsm',
       'national_lidar_programme_intensity',
       'national_lidar_programme_point_cloud',
       'national_lidar_programme_vom', 'surfzone_dem_2019',
       'vertical_aerial_photography_tiles_rgbn',
       'vertical_aerial_photography_tiles_irrgbn',
       'oblique_aerial_photography_tiles_incident_response',
       'vertical_aerial_photography_tiles_rgb',
       'vertical_aerial_photography_tiles_irrgb',
       'bathymetry_coastal_multibeam',
       'oblique_aerial_photography_tiles_other_survey',
       'bathymetry_riverine_multibeam',
       'vertical_aerial_photography_tiles_night_time'], dtype=object)

In [4]:
product_counts = gpd.GeoDataFrame(products.groupby(["product_id", "geometry"]).size().reset_index().rename(columns={0: "count"}).sort_values("count", ascending=False))

In [7]:
product_counts[product_counts.product_id == "vertical_aerial_photography_tiles_irrgb"].explore("count")

In [9]:
products.iloc[1].uri

'https://api.agrimetrics.co.uk/tiles/collections/survey/lidar_composite_dtm/2022/1/NU0050'

In [13]:
def append_key(uri):
    return f"{uri}?subscription-key=public"

append_key(products[products.product_id == "vertical_aerial_photography_tiles_irrgb"].iloc[0].uri)

'https://api.agrimetrics.co.uk/tiles/collections/survey/vertical_aerial_photography_tiles_irrgb/2009/0.2/NX9525?subscription-key=public'

In [15]:
import requests
from tqdm import tqdm
def download_image(url: str, file_path: str):
    response = requests.get(url, stream=True)
    with open(file_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=2048):
            file.write(chunk)
    return file_path

product_uri = append_key(products[products.product_id == "vertical_aerial_photography_tiles_irrgb"].iloc[10].uri)
print("Downloading: ", product_uri)

download_image(product_uri, "data/test.zip")


Downloading:  https://api.agrimetrics.co.uk/tiles/collections/survey/vertical_aerial_photography_tiles_irrgb/2009/0.1/NT9520?subscription-key=public


'data/test.zip'

In [58]:
import folium
import pyproj

m = folium.Map(location=[52.52, 13.4050], zoom_start=10)
bounds_3857 = (-191092.570712924, 6625485.174730781, -183066.68274298048, 6633511.062700724)
xmin, ymin = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform(bounds_3857[0], bounds_3857[1])
xmax, ymax = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform(bounds_3857[2], bounds_3857[3])


tiles_url = "http://127.0.0.1:8000/tile/{z}/{x}/{y}"
folium.TileLayer(tiles=tiles_url, attr="test").add_to(m)
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
# add layer control
folium.LayerControl().add_to(m)
m

